<a href="https://colab.research.google.com/github/KarelZe/adv-ml/blob/main/AML_Bonus_MarkusBilz_2327197_VerenaHollinger_2055134.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [35]:
#import dependencies
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split


# Task 1: Explain dataset and business problems in two paragraphs. (10 points)
...

## Loading the data

In [ ]:
import pandas as pd
from google.colab import drive


drive.mount("/content/drive")

data = pd.read_csv('/content/drive/My Drive/data/accepted_2007_to_2018Q4.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


FileNotFoundError: ignored

In [ ]:
import pandas as pd
from google.colab import drive

drive.mount("/content/drive")

data = pd.read_csv('/content/drive/My Drive/AdvancedML/accepted_2007_to_2018Q4.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# Task 2: Do an explanatory analysis for this dataset. (10 points)

In [ ]:
data.head().T

In [ ]:
a_priori_known = ['id', 'loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate', 'installment', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'annual_inc', 'verification_status', 'issue_d', 'loan_status', 'purpose', 'title', 'zip_code', 'addr_state', 'dti', 'delinq_2yrs', 'earliest_cr_line', 'fico_range_low', 'fico_range_high', 'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'initial_list_status',  'next_pymnt_d', 'last_credit_pull_d', 'last_fico_range_high', 'last_fico_range_low', 'collections_12_mths_ex_med', 'mths_since_last_major_derog', 'policy_code', 'application_type', 'annual_inc_joint', 'dti_joint', 'verification_status_joint', 'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'open_acc_6m', 'open_act_il', 'open_il_12m', 'open_il_24m', 'mths_since_rcnt_il', 'total_bal_il', 'il_util', 'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_rev_hi_lim', 'inq_fi', 'total_cu_tl', 'inq_last_12m', 'acc_open_past_24mths', 'avg_cur_bal', 'bc_open_to_buy', 'bc_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'mo_sin_old_il_acct', 'mo_sin_old_rev_tl_op', 'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc',  'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_bc_sats', 'num_bc_tl', 'num_il_tl', 'num_op_rev_tl', 'num_rev_accts', 'num_rev_tl_bal_gt_0', 'num_sats', 'num_tl_120dpd_2m', 'num_tl_30dpd', 'num_tl_90g_dpd_24m', 'num_tl_op_past_12m', 'pct_tl_nvr_dlq', 'percent_bc_gt_75', 'pub_rec_bankruptcies', 'tax_liens', 'tot_hi_cred_lim', 'total_bal_ex_mort', 'total_bc_limit', 'total_il_high_credit_limit', 'revol_bal_joint']

data = data[a_priori_known]

In [ ]:
threshold = 0.80

percentage_missing_val = data.isna().sum()/len(data)
percentage_missing_val = percentage_missing_val[percentage_missing_val >= threshold]

print(percentage_missing_val)
threshold_abs = int(threshold * len(data))

data = data.dropna(axis=1, thresh=threshold_abs)

print(data.shape)

In [ ]:
data = data[data.loan_amnt.notna()]

Hier hatte ich noch verification status und id ergänzt und purpose, emp_length und term zu den categorical gemacht
addr_state hätte 51 Klassen, viel für OneHotEncoding, aber können wir es sonst überhaupt verwenden?

In [ ]:
# convert to categorical
categorical = ['term', 'application_type', 'initial_list_status', 'loan_status', 'grade', 'sub_grade', 'home_ownership', 'verification_status', 'purpose','emp_length']
data[categorical] = data[categorical].apply(lambda x: x.astype('category'))

# convert to string
strs = ['emp_title', 'title', 'zip_code', 'addr_state']
data[strs] = data[strs].apply(lambda x: x.astype('string'))

# convert to datetime
dts = ['earliest_cr_line', 'issue_d', 'last_credit_pull_d']
data[dts] = data[dts].apply(lambda x: pd.to_datetime(x, format="%b-%Y",  errors='coerce'))

# convert to integer
ints = ['id']
data[ints] = data[ints].apply(lambda x: x.astype('int'))

print(data.dtypes)

##Number of classes per categorical feature

In [ ]:
data_classes = []
for row in data.columns.tolist():
    data_classes.append({'column': row, 'count': len(data[row].unique())})
amount = pd.DataFrame(data_classes).sort_values('count')
print(amount)

###The features "purpose" and "title" contain similar content. Since "purpose" only contains 14 classes and "title" contains more than 60,000 classes we keep "purpose" and drop "title"

In [ ]:
data = data.drop(columns=['title'])

##Analyze loan status

In [ ]:
data['loan_status'].value_counts()

In [ ]:
target_list = [1 if i=='Default' or i=='Charged Off' or i=='Does not meet the credit policy. Status:Charged Off' else 0 for i in data['loan_status']]
data['TARGET'] = target_list
data['TARGET'].value_counts()

## Data Preprocessing

Since the features have different units, Scaling and Standardization of the numeric features are necessary

In [ ]:
data_num = data.select_dtypes(include=[np.float64, np.float32])
data_num_columns = data_num.columns
data = data.drop(columns=data_num_columns)
min_max = MinMaxScaler()
data_scaled = pd.DataFrame(min_max.fit_transform(data_num), columns = data_num_columns)

data_scaled.describe()

In [ ]:
standardizer = StandardScaler()
data_standardized = pd.DataFrame(standardizer.fit_transform(data_scaled), columns = data_num_columns)
data_standardized.describe()

All categorical values ar binarized using get_dummies

In [ ]:
data_cat = data.select_dtypes(include=['category'])
data_cat_columns = data_cat.columns
data_cat_columns

In [ ]:
data = data.drop(columns=data_cat_columns)
data_to_encode = data_cat.drop(columns=['loan_status'])
data_encoded = pd.get_dummies(data_to_encode)
data_encoded

If there are only two classes, they have a correlation of 1, so one class can be dropped without loosing any information

In [ ]:
data_encoded = data_encoded.drop(columns=['term_ 60 months', 'initial_list_status_w', 'application_type_Joint App'])
data_cat

In [ ]:
target = data[['TARGET']]
data = data.drop(columns=['TARGET'])
data = pd.concat([data, data_standardized, data_cat], axis=1)
data[['Target']] = target
data

In [ ]:
#data_to_encode = data[['term', 'verification_status','application_type']]
#enc = OneHotEncoder()
#df_num = enc.fit_transform(data_to_encode).toarray()
#df_num = pd.DataFrame(df_num)
#df_num

To identify similar features and relevant features (high correlation with target feature) a correlation analysis is conducted

In [ ]:
#correlation with target feature
corr = data.corr()['TARGET'].sort_values()
print('Most Positive Correlations:\n',corr.tail(15))
print('\nMost Negative Correlations:\n',corr.head(15))

## Task 3: Use LASSO for variable selection and explain results. (15 points)

## Task 4: Use logistic regression (in-sample) for default prediction. Interpret your results for logistic regression. (20 points)

## Task 5: Compare logistic regression, classification tree, random forest, bagging, support vector machine (with two kernel functions), and neural network for default prediction based on the out-of-sample performance measures (at least three different measures). (35 points)

## Task 6: Which model do you suggest to the peer-to-peer company? Why? (10 points)